In [2]:
!pip install deepface
!pip numpy
!pip pandas
!pip opencv-python
!pip matplotlib
!pip tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c84eb84db6099f5746ce08ea7db8a7a688d507344eee800c88eccd5a7a817487
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
ERROR: unknown command "numpy"
ERROR: unknown command "pandas"
ERROR: unknown command "opencv-python"
ERROR: unknown command "matplotlib"
ERROR: unknown command "tf-keras"


In [ ]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

In [ ]:
# List of available backends, models, and distance metrics
backends = ["opencv", "ssd", "dlib", "mtcnn", "retinaface"]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
metrics = ["cosine", "euclidean", "euclidean_l2"]

In [ ]:
import zipfile
with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
  zip_ref.extractall('Data')

In [ ]:
# Path to the image for face recognition
img = "Data/Data/klaus.png"
output_folder = "matched_images"
db_path = 'Data/Data/'

os.makedirs(output_folder, exist_ok=True)

def face_recognition(img):
    # Perform face recognition on the provided image
    # Find faces and identify people using a specific model and distance metric
    people = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[1])

    # Display the original image
    # plt.imshow(cv2.imread(img))

    # Print the identities of the recognized people
    for person in people:
      print(f'------------------ {person}')
      print(person['identity'][0].split('/')[1])


    for i, person_df in enumerate(people):
        print(f'-------- Matches for face {i + 1}: {len(person_df)} match(es) found --------')
        for index, row in person_df.iterrows():
            identity_path = row['identity']
            print(identity_path)


def face_recognition_v2(img):
    # Perform face recognition using DeepFace
    results = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[1])

    # Get base name of query image (e.g., "klaus.png")
    query_image_name = os.path.basename(img)

    # results is a list of DataFrames (one for each face found)
    for df in results:
        for _, row in df.iterrows():
            matched_path = row['identity']
            matched_filename = os.path.basename(matched_path)

            # Skip the query image itself
            if matched_filename == query_image_name:
                continue

            # Copy matched file to output folder
            destination_path = os.path.join(output_folder, matched_filename)
            shutil.copy(matched_path, destination_path)
            print(f"Matched and copied: {matched_filename}")

# Perform face recognition on a single image
face_recognition_v2(img)

25-05-30 12:10:46 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:04<00:00,  4.84s/it]


25-05-30 12:10:51 - There are now 14 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
25-05-30 12:10:51 - Searching Data/Data/klaus.png in 14 length datastore
25-05-30 12:10:52 - find function duration 6.3281872272491455 seconds
Matched and copied: klaus_family.png


In [ ]:
def realtime_face_recognition():
    # Define a video capture object
    vid = cv2.VideoCapture(0)

    while True:
        # Capture the video frame by frame
        ret, frame = vid.read()

        # Perform face recognition on the captured frame
        # Find faces and identify people using a specific model and distance metric
        people = DeepFace.find(img_path=frame, db_path=db_path, model_name=models[2], distance_metric=metrics[2], enforce_detection=False)

        for person in people:
            # Retrieve the coordinates of the face bounding box
            x = person['source_x'][0]
            y = person['source_y'][0]
            w = person['source_w'][0]
            h = person['source_h'][0]

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Get the person's name and display it on the image
            name = person['identity'][0].split('/')[1]
            cv2.putText(frame, name, (x, y), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Display the resulting frame
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 960, 720)
        cv2.imshow('frame', frame)

        # Check if the 'q' button is pressed to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    vid.release()
    cv2.destroyAllWindows()

# Perform real-time face recognition using the webcam
# realtime_face_recognition()

In [6]:
import os
import cv2
import numpy as np
from PIL import Image, ExifTags
from deepface import DeepFace

output_folder = 'matched_output'

models = ['Facenet', 'Facenet512', 'OpenFace', 'DeepFace', 'DeepID', 'Dlib', 'ArcFace', 'SFace' 'GhostFaceNet']
distance_metrics = ['cosine', 'euclidean', 'euclidean_l2']
detectorbackend = ['opencv', 'retinaface', 'mtcnn', 'ssd', 'dlib', 'mediapipe', 'yolov8', 'centerface']

# ==================== Working Script ============================
def load_image_with_correct_orientation(img_path):
    try:
        image = Image.open(img_path)

        # Apply EXIF rotation if available
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break

        exif = image._getexif()
        if exif is not None:
            orientation_value = exif.get(orientation)
            if orientation_value == 3:
                image = image.rotate(180, expand=True)
            elif orientation_value == 6:
                image = image.rotate(270, expand=True)
            elif orientation_value == 8:
                image = image.rotate(90, expand=True)

        # Convert to OpenCV format
        image_cv2 = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        return image_cv2
    except Exception as e:
        print("EXIF rotation failed:", e)
        return cv2.imread(img_path)


def face_recognition_v2_optimized(img):
    db_path = '/home/urvishbhatt/Github Projects/Face_Recognition_DeepFace/face-reco-v2/album_v2'
    model_name = "Facenet512"  # models[2]
    distance_metric = "euclidean_l2"  # metrics[2]
    max_threshold = 0.7

    faces = DeepFace.extract_faces(img_path=img, detector_backend="retinaface")

    # Use faster detector for better performance
    results = DeepFace.find(
        img_path=img,
        db_path=db_path,
        model_name=model_name,
        distance_metric='euclidean_l2',
        enforce_detection=True,
        expand_percentage=55,
        threshold=0.90,
        normalization='Facenet',
        detector_backend="retinaface"
    )
    print("results:", results)

    query_image_name = os.path.basename(img)

    for df in results:

        for _, row in df.iterrows():

            if row["distance"] > max_threshold:
                continue  # Skip weak matches

            matched_path = row["identity"]
            matched_filename = os.path.basename(matched_path)

            if matched_filename == query_image_name:
                continue

            # destination_path = os.path.join(output_folder, matched_filename)
            # if not os.path.exists(destination_path):  # Avoid duplicate copies
            #     shutil.copy(matched_path, destination_path)
            #     print(f"Matched and copied: {matched_filename} (Distance: {row['distance']:.4f})")
            #     matches_found += 1
            # img = cv2.imread(matched_path)
            img = load_image_with_correct_orientation(matched_path)

            print("img:", img)

            # bounding-box in the TARGET image
            x, y = int(row["target_x"]), int(row["target_y"])
            w, h = int(row["target_w"]), int(row["target_h"])

            # green rectangle + distance label
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img,
                        f"{row['distance']:.3f}",
                        (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0, 255, 0),
                        2)

            out_name = "boxed_" + os.path.basename(matched_path)
            cv2.imwrite(os.path.join(output_folder, out_name), img)


# face_recognition_v2_optimized("KRA09267.jpeg")
face_recognition_v2_optimized("KRA09313.jpeg")

25-06-05 07:31:05 - Searching imgs/crop_img.png in 135 length datastore
25-06-05 07:31:05 - find function duration 0.37215685844421387 seconds
results: [               identity                                      hash  target_x  \
0    Data/KRA09302.jpeg  cc705e0a724a641cc3147c59ac877af9d02e0ca8      2419   
1    Data/KRA09303.jpeg  8cae3bbd0251cc07c354039393cadad31ddf645e      2740   
2    Data/KRA09301.jpeg  500c215bea683cb34843cba5f99ce860e9c5646d      2624   
3    Data/KRA09317.jpeg  b1a1b9b51ee843efb3c5dcbe8bf43f7fea41835f      6714   
4    Data/KRA09302.jpeg  cc705e0a724a641cc3147c59ac877af9d02e0ca8      3200   
..                  ...                                       ...       ...   
125  Data/KRA09314.jpeg  ff38985f0601c9d4f2b5239cf2ea7bf3c1c9d345       767   
126  Data/KRA09302.jpeg  cc705e0a724a641cc3147c59ac877af9d02e0ca8         0   
127  Data/KRA09319.jpeg  c0e884b2b4eaa448f0b537c7e2f0615cfeef2107      3675   
128  Data/KRA09309.jpeg  639defb60132a28faf2538e1c9d0794d